## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [4]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [5]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_df.head(100)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,Hithadhoo,MV,82.11,scattered clouds,-0.6000,73.0833,Scoop Guest House
1,5,Saint-Joseph,RE,77.36,scattered clouds,-21.3667,55.6167,"""Plantation Bed and Breakfast"
2,7,Meulaboh,ID,78.24,overcast clouds,4.1363,96.1285,Kost Pria
3,8,Coihaique,CL,70.41,scattered clouds,-45.5752,-72.0662,Diego De Almagro Coyhaique
4,10,Rikitea,PF,78.12,overcast clouds,-23.1203,-134.9692,People ThankYou
5,11,Vaini,TO,73.56,light intensity shower rain,-21.2000,-175.2000,Keleti Beach Resort
6,14,Butaritari,KI,76.57,very heavy rain,3.0707,172.7902,Isles Sunset Lodge
7,15,Sibolga,ID,76.21,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
8,20,Dingle,PH,73.33,clear sky,10.9995,122.6711,SEGAYA PROPERTY
9,21,Kalmunai,LK,77.68,overcast clouds,7.4167,81.8167,VS Villa


In [6]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [7]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 4b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Vaini"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Vaini"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Saint-Joseph"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Sibolga"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Georgetown"] 

In [9]:
print (vacation_start)

   City_ID   City Country  Max Temp          Current Description   Lat    Lng  \
5       11  Vaini      TO     73.56  light intensity shower rain -21.2 -175.2   

            Hotel Name  
5  Keleti Beach Resort  


In [10]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0],vacation_end["Lng"].to_numpy()[0]
print(f'Start: {start}, End: {end}')

stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]
print(f'Stop 1: {stop1}, Stop 2: {stop2}, Stop 3:{stop3}')


Start: (-21.2, -175.2), End: (-21.2, -175.2)
Stop 1: (-21.3667, 55.6167), Stop 2: (1.7427, 98.7792), Stop 3:(5.4112, 100.3354)


In [11]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
road_route = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2,stop3],
       travel_mode='DRIVING')
fig.add_layer(road_route)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, 
                          vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,11,Vaini,TO,73.56,light intensity shower rain,-21.2000,-175.2000,Keleti Beach Resort
1,5,Saint-Joseph,RE,77.36,scattered clouds,-21.3667,55.6167,"""Plantation Bed and Breakfast"
2,15,Sibolga,ID,76.21,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
3,39,Georgetown,MY,82.89,few clouds,5.4112,100.3354,Cititel Penang


In [13]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(road_route)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))